In [1]:
import csv
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

# 1) Importing drug data into a df for clean-up

In [2]:
df = pd.read_csv('C:/Users/New User/Desktop/Metis/NLP/drugsCom_raw/drugs_clean_data.csv')

In [3]:
df

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,strlen,tokens,text
0,0,Valsartan,Left Ventricular Dysfunction,it has no side effect i take it in combination...,9,20-May-12,27,16,"['it', 'has', 'no', 'side', 'effect', 'i', 'ta...",it has no side effect i take it in combination...
1,1,Guanfacine,ADHD,my son is halfway through his fourth week of i...,8,27-Apr-10,192,140,"['my', 'son', 'is', 'halfway', 'through', 'his...",my son is halfway through his fourth week of i...
2,2,Lybrel,Birth Control,i used to take another oral contraceptive whic...,5,14-Dec-09,17,132,"['i', 'used', 'to', 'take', 'another', 'oral',...",i used to take another oral contraceptive whic...
3,3,Ortho Evra,Birth Control,this is my first time using any form of birth ...,8,3-Nov-15,10,86,"['this', 'is', 'my', 'first', 'time', 'using',...",this is my first time using any form of birth ...
4,4,Buprenorphine / naloxone,Opiate Dependence,suboxone has completely turned my life around ...,9,27-Nov-16,37,125,"['suboxone', 'has', 'completely', 'turned', 'm...",suboxone has completely turned my life around ...
...,...,...,...,...,...,...,...,...,...,...
212845,212845,Tamoxifen,"Breast Cancer, Prevention",i have taken tamoxifen for years side effects ...,10,13-Sep-14,43,93,"['i', 'have', 'taken', 'tamoxifen', 'for', 'ye...",i have taken tamoxifen for years side effects ...
212846,212846,Escitalopram,Anxiety,ive been taking lexapro escitaploprgram since ...,9,8-Oct-16,11,132,"['ive', 'been', 'taking', 'lexapro', 'escitapl...",ive been taking lexapro escitaploprgram since ...
212847,212847,Levonorgestrel,Birth Control,im married years old and i have no kids taking...,8,15-Nov-10,7,146,"['im', 'married', 'years', 'old', 'and', 'i', ...",im married years old and i have no kids taking...
212848,212848,Tapentadol,Pain,i was prescribed nucynta for severe neckshould...,1,28-Nov-11,20,33,"['i', 'was', 'prescribed', 'nucynta', 'for', '...",i was prescribed nucynta for severe neckshould...


In [4]:
# There are 200k rows and 10 columns. Dropping all columns except for 'drugName', 'condition', 'rating' to import into SQLite

In [5]:
df = df[['drugName', 'condition', 'rating']]

In [6]:
df

,drugName,condition,rating
0,Valsartan,Left Ventricular Dysfunction,9
1,Guanfacine,ADHD,8
2,Lybrel,Birth Control,5
3,Ortho Evra,Birth Control,8
4,Buprenorphine / naloxone,Opiate Dependence,9
...,...,...,...
212845,Tamoxifen,"Breast Cancer, Prevention",10
212846,Escitalopram,Anxiety,9
212847,Levonorgestrel,Birth Control,8
212848,Tapentadol,Pain,1


In [7]:
df.describe()

,rating
count,212850.000000
mean,6.990933
std,3.275255
min,1.000000
25%,5.000000
50%,8.000000
75%,10.000000
max,10.000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212850 entries, 0 to 212849
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   drugName   212850 non-null  object
 1   condition  212850 non-null  object
 2   rating     212850 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 4.9+ MB


In [9]:
df.nunique()

drugName     3665
condition     916
rating         10
dtype: int64

In [10]:
df.condition.unique()

array(['Left Ventricular Dysfunction', 'ADHD', 'Birth Control',
       'Opiate Dependence', 'Benign Prostatic Hyperplasia',
       'Emergency Contraception', 'Bipolar Disorde', 'Epilepsy',
       'Migraine Prevention', 'Depression', "Crohn's Disease", 'Cough',
       'Obesity', 'Urinary Tract Infection', 'ibromyalgia',
       'Chronic Myelogenous Leukemia', 'HIV Infection', 'Insomnia',
       'Rheumatoid Arthritis', 'Vaginal Yeast Infection',
       'Chlamydia Infection', 'Hirsutism', 'Panic Disorde', 'Migraine',
       'Pain', 'Osteoarthritis', 'Constipation', 'Bowel Preparation',
       'Psychosis', 'Muscle Spasm', 'Hepatitis C', 'Overactive Bladde',
       'Diabetes, Type 2', 'Asthma, Maintenance',
       'Non-Small Cell Lung Cance', 'Dysuria', 'Smoking Cessation',
       'Anxiety', 'Acne', 'emale Infertility', 'Constipation, Acute',
       'Constipation, Drug Induced', 'Erectile Dysfunction',
       'Trigeminal Neuralgia', 'Underactive Thyroid', 'Chronic Pain',
       'Atrophic Vag

In [11]:
df.to_csv('C:/Users/New User/Desktop/drugs_sql.csv')

# 2) Importing 'drugs_sql.csv' as a database into SQLite via DBrowser

# 3) Querying into 'drug_db' created in DBrowser

In [12]:
engine = create_engine("sqlite:///C:/Users/New User/Desktop/drugdb.db")


In [13]:
insp = inspect(engine)
print(insp.get_table_names())

['drug_db']


In [14]:
drug_data = pd.read_sql('SELECT * FROM drug_db;', engine)
drug_data

,field1,drugName,condition,rating
0,0,Valsartan,Left Ventricular Dysfunction,9
1,1,Guanfacine,ADHD,8
2,2,Lybrel,Birth Control,5
3,3,Ortho Evra,Birth Control,8
4,4,Buprenorphine / naloxone,Opiate Dependence,9
...,...,...,...,...
212845,212845,Tamoxifen,"Breast Cancer, Prevention",10
212846,212846,Escitalopram,Anxiety,9
212847,212847,Levonorgestrel,Birth Control,8
212848,212848,Tapentadol,Pain,1


In [26]:
drug_data = pd.read_sql('SELECT RANK()OVER(PARTITION BY condition, drugName ORDER BY rating DESC) AS Num, condition, drugName, rating FROM drug_db;', engine)
drug_data

,Num,condition,drugName,rating
0,1,0</span> users found this comment helpful.,Aricept,2
1,1,0</span> users found this comment helpful.,Atripla,8
2,1,0</span> users found this comment helpful.,Aviane,7
3,1,0</span> users found this comment helpful.,Aviane,7
4,3,0</span> users found this comment helpful.,Aviane,1
...,...,...,...,...
212845,3,zen Shoulde,Naproxen,1
212846,1,zen Shoulde,Relafen,9
212847,2,zen Shoulde,Relafen,7
212848,1,zen Shoulde,Voltaren,8


In [27]:
drug_rankings_df = drug_data

In [28]:
drug_rankings_df.rename(columns={'Num':'rank'}, inplace=True)

In [29]:
drug_rankings_df

,rank,condition,drugName,rating
0,1,0</span> users found this comment helpful.,Aricept,2
1,1,0</span> users found this comment helpful.,Atripla,8
2,1,0</span> users found this comment helpful.,Aviane,7
3,1,0</span> users found this comment helpful.,Aviane,7
4,3,0</span> users found this comment helpful.,Aviane,1
...,...,...,...,...
212845,3,zen Shoulde,Naproxen,1
212846,1,zen Shoulde,Relafen,9
212847,2,zen Shoulde,Relafen,7
212848,1,zen Shoulde,Voltaren,8


In [30]:
drug_rankings_df.to_csv('C:/Users/New User/Desktop/drug_rankings.csv')

# 4) After drugs are ranked in SQL, they are saved in a 'drug_rankings.csv'

# 5) To make the dataset compact and robust for Streamlit, drugs with low rankings are removed and only drugs with high rankings are kept in the dataset

In [31]:
# creating top_drugs_df
top_drugs_mask = drug_rankings_df.rating > 7
top_drugs_df = drug_rankings_df[top_drugs_mask]
top_drugs_df

,rank,condition,drugName,rating
1,1,0</span> users found this comment helpful.,Atripla,8
7,1,0</span> users found this comment helpful.,Chantix,9
8,2,0</span> users found this comment helpful.,Chantix,8
12,1,0</span> users found this comment helpful.,Depo-Provera,10
13,2,0</span> users found this comment helpful.,Depo-Provera,9
...,...,...,...,...
212837,1,zen Shoulde,Indocin,10
212838,1,zen Shoulde,Indomethacin,10
212839,1,zen Shoulde,Nabumetone,9
212846,1,zen Shoulde,Relafen,9


In [33]:
top_drugs_df.nunique()

rank          279
condition     838
drugName     3366
rating          3
dtype: int64

In [34]:
top_drugs_df.to_csv('C:/Users/New User/Desktop/drug_top.csv')

# 6) Drug Recommendation Program in Python

In [35]:
import streamlit as st

In [36]:
top_drugs_df.head()

,rank,condition,drugName,rating
1,1,0</span> users found this comment helpful.,Atripla,8
7,1,0</span> users found this comment helpful.,Chantix,9
8,2,0</span> users found this comment helpful.,Chantix,8
12,1,0</span> users found this comment helpful.,Depo-Provera,10
13,2,0</span> users found this comment helpful.,Depo-Provera,9


In [37]:
df = top_drugs_df

In [ ]:

# def drug_rec(user_condition):
#     def get_index_from_condition(condition):
#         return df[df.condition == condition]['index'].values[0]

#     def get_drug_from_index(index):
#         return df[df.index == index]['drugName'].values[0]

# #     cv=CountVectorizer()
# #     count_matrix=cv.fit_transform(df)
# #     cosine_sim=cosine_similarity(count_matrix)
    
#     condition_index = get_index_from_condition(user_condition)
#     similar_drugs=list(enumerate(cosine_sim[condition_index]))
#     sorted_similar_drugs = sorted(similar_drugs, key=lambda x: x[1], reverse=True)
#     i=0
#     for drug in sorted_similar_drugs:
#         print (get_drug_from_index(drug[0]))
#         i=i+1
#         if i>5:
#             break
#     return get_drug_from_index(drug[0])

In [38]:
df

,rank,condition,drugName,rating
1,1,0</span> users found this comment helpful.,Atripla,8
7,1,0</span> users found this comment helpful.,Chantix,9
8,2,0</span> users found this comment helpful.,Chantix,8
12,1,0</span> users found this comment helpful.,Depo-Provera,10
13,2,0</span> users found this comment helpful.,Depo-Provera,9
...,...,...,...,...
212837,1,zen Shoulde,Indocin,10
212838,1,zen Shoulde,Indomethacin,10
212839,1,zen Shoulde,Nabumetone,9
212846,1,zen Shoulde,Relafen,9


In [74]:
df = df.drop_duplicates(subset=['drugName', 'condition'], keep='first')

In [75]:
df

,rank,condition,drugName,rating
1,1,0</span> users found this comment helpful.,Atripla,8
7,1,0</span> users found this comment helpful.,Chantix,9
12,1,0</span> users found this comment helpful.,Depo-Provera,10
20,1,0</span> users found this comment helpful.,Drysol,10
25,1,0</span> users found this comment helpful.,GoLYTELY,8
...,...,...,...,...
212837,1,zen Shoulde,Indocin,10
212838,1,zen Shoulde,Indomethacin,10
212839,1,zen Shoulde,Nabumetone,9
212846,1,zen Shoulde,Relafen,9


In [93]:
top_drugs = []

def drug_rec(user_condition):
    for condition in df.condition:
        if user_condition == condition:
            top_drugs.append(df.drugName)
    return top_drugs  

In [94]:
drug_rec('Birth Control')[:1]

[1              Atripla
 7              Chantix
 12        Depo-Provera
 20              Drysol
 25            GoLYTELY
               ...     
 212837         Indocin
 212838    Indomethacin
 212839      Nabumetone
 212846         Relafen
 212848        Voltaren
 Name: drugName, Length: 8124, dtype: object]

In [96]:
df = pd.read_csv('C:/Users/New User/Desktop/top_drugs.csv')

In [97]:
df

,reviews,drugName,condition,rating,rank
0,1,A + D Cracked Skin Relief,Bacterial Skin Infection,10,1
1,2,A / B Otic,Otitis Media,10,1
2,2,A / B Otic,Otitis Media,10,1
3,52,Abacavir / dolutegravir / lamivudine,HIV Infection,10,1
4,52,Abacavir / dolutegravir / lamivudine,HIV Infection,10,1
...,...,...,...,...,...
128446,45,ella,Emergency Contraception,8,37
128447,45,ella,Emergency Contraception,8,37
128448,45,ella,Emergency Contraception,8,37
128449,45,ella,Emergency Contraception,8,37


In [127]:
top_drugs = []

def drug_rec(user_condition):
    for condition in df.condition:
        if user_condition == condition:
            top_drugs.append(df.drugName[:3])
    return top_drugs

In [128]:
drug_rec('Depression')[:1]

[0    A + D Cracked Skin Relief
 1                   A / B Otic
 2                   A / B Otic
 Name: drugName, dtype: object]

In [ ]:
# SELECT COUNT(*)OVER(PARTITION BY drugName) AS reviews, drugName, condition, rating, rank FROM drug_top WHERE condition= 'Acne' ORDER BY rating DESC;

In [138]:
engine = create_engine("sqlite:///C:/Users/New User/Desktop/drugdb.db")

In [139]:
insp = inspect(engine)
print(insp.get_table_names())

['drug_db', 'drug_top']


In [157]:
user_condition = '"Acne"'

In [152]:
user_condition = '"Depression"'

In [164]:
user_condition = '"HIV Infection"'

In [165]:
drug_results = pd.read_sql('SELECT COUNT(*)OVER(PARTITION BY drugName) AS reviews, drugName, condition, rating, rank FROM drug_top WHERE condition= '+user_condition+' ORDER BY rating DESC;', engine)
drug_results

,reviews,drugName,condition,rating,rank
0,52,Abacavir / dolutegravir / lamivudine,HIV Infection,10,1
1,52,Abacavir / dolutegravir / lamivudine,HIV Infection,10,1
2,52,Abacavir / dolutegravir / lamivudine,HIV Infection,10,1
3,52,Abacavir / dolutegravir / lamivudine,HIV Infection,10,1
4,52,Abacavir / dolutegravir / lamivudine,HIV Infection,10,1
...,...,...,...,...,...
657,71,Stribild,HIV Infection,8,69
658,1,Sustiva,HIV Infection,8,1
659,8,Tivicay,HIV Infection,8,8
660,51,Triumeq,HIV Infection,8,51


In [166]:
drug_results.drop_duplicates()
drug_results[:3]

,reviews,drugName,condition,rating,rank
0,52,Abacavir / dolutegravir / lamivudine,HIV Infection,10,1
1,52,Abacavir / dolutegravir / lamivudine,HIV Infection,10,1
2,52,Abacavir / dolutegravir / lamivudine,HIV Infection,10,1


In [167]:
drug_results = drug_results.drop_duplicates()

In [168]:
drug_results

,reviews,drugName,condition,rating,rank
0,52,Abacavir / dolutegravir / lamivudine,HIV Infection,10,1
37,3,Abacavir / lamivudine,HIV Infection,10,1
40,1,Atazanavir,HIV Infection,10,1
41,81,Atripla,HIV Infection,10,1
88,1,Cobicistat / darunavir,HIV Infection,10,1
...,...,...,...,...,...
655,71,Stribild,HIV Infection,8,69
658,1,Sustiva,HIV Infection,8,1
659,8,Tivicay,HIV Infection,8,8
660,51,Triumeq,HIV Infection,8,51


In [169]:
drug_results[:3]

,reviews,drugName,condition,rating,rank
0,52,Abacavir / dolutegravir / lamivudine,HIV Infection,10,1
37,3,Abacavir / lamivudine,HIV Infection,10,1
40,1,Atazanavir,HIV Infection,10,1


# 7) Complete Drug Recommendation Program for Streamlit

In [170]:
def drug_rec(user_condition):
    engine = create_engine("sqlite:///C:/Users/New User/Desktop/drugdb.db")
    drug_results = pd.read_sql('SELECT COUNT(*)OVER(PARTITION BY drugName) AS reviews, drugName, condition, rating, rank FROM drug_top WHERE condition= '+user_condition+' ORDER BY rating DESC;', engine)
    drug_results = drug_results.drop_duplicates()
    print(drug_results[:3])

In [171]:
user_condition = '"Acne"'
drug_rec(user_condition)

    reviews  drugName condition  rating  rank
0         2  Absorica      Acne      10     1
1        38    Acanya      Acne      10     1
19      339  Accutane      Acne      10     1


In [172]:
user_condition = '"Depression"'
drug_rec(user_condition)

     reviews       drugName   condition  rating  rank
0        115        Abilify  Depression      10     1
58        71     Alprazolam  Depression      10     1
115       42  Amitriptyline  Depression      10     1


In [173]:
user_condition = '"HIV Infection"'
drug_rec(user_condition)

    reviews                              drugName      condition  rating  rank
0        52  Abacavir / dolutegravir / lamivudine  HIV Infection      10     1
37        3                 Abacavir / lamivudine  HIV Infection      10     1
40        1                            Atazanavir  HIV Infection      10     1


In [177]:
user_condition = '"Schizophrenia"'
drug_rec(user_condition)

    reviews          drugName      condition  rating  rank
0        22           Abilify  Schizophrenia      10     1
11        4  Abilify Maintena  Schizophrenia      10     1
15       28      Aripiprazole  Schizophrenia      10     1


In [178]:
import streamlit as st

In [179]:
st.title("PeopleRX: A Drug Recommendation System For People By The People")

st.header("This system recommends top 3 most effective medications for a chosen medical condition")

st.subheader("Enter a medical condition:")
st.selectbox("Condition", condition, index=1)

if st.button("Recommend Me:"):
    get_drugs()


st.text("Disclaimer: \nThis system and its recommendations are not official medical advice. \nPlease use at your own risk.")


2022-08-06 12:10:54.237 
  command:

    streamlit run C:\Users\New User\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


NameError: name 'condition' is not defined